In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Download Kepler PDCSAP light curves for KIC 3975085, segment by time gaps,
detrend each segment with a 3rd-order polynomial, and save time-normalized flux.
Output file: "KIC 3975085 flux 2.txt"
"""

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt  

# --- 1) download Kepler light curves (PDCSAP) ---
lc_K = lk.search_lightcurve("kic 3975085", author="Kepler", cadence="long").download_all()

# --- 2) collect time / flux / mag (keep only positive pdcsap_flux) ---
time = []
flux = []
mag  = []

for i in range(len(lc_K) - 2):
    # keep the loop and condition exactly as in the original code
    for j in range(len(lc_K[i])):
        val = lc_K[i][j]["pdcsap_flux"].value.item()
        if val > 0:
            t = lc_K[i][j]["time"].value
            time.append(t)
            mag.append(25 - 2.5 * np.log10(val))
            flux.append(val)

time = np.array(time)
flux = np.array(flux)
mag  = np.array(mag)

# --- 3) sort by time (apply same ordering to all arrays) ---
sort_idx = np.argsort(time)
time = time[sort_idx]
mag  = mag[sort_idx]
flux = flux[sort_idx]

# --- 4) split by time gaps (> 0.2 days) ---
threshold = 0.2
split_indices = np.where(np.diff(time) > threshold)[0] + 1
split_indices = list(split_indices)
time_subsequences = np.split(time, split_indices)
mag_subsequences  = np.split(mag,  split_indices)
flux_subsequences = np.split(flux, split_indices)

# --- 5) per-segment polynomial detrend (degree=3), then stitch back ---
fit = []
for i in range(len(time_subsequences)):
    degree = 3
    coefficients = np.polyfit(time_subsequences[i], flux_subsequences[i], degree)
    poly_fit = np.poly1d(coefficients)
    tmp_fit = poly_fit(time_subsequences[i])
    for j in range(len(tmp_fit)):
        fit.append(tmp_fit[j])

fit = np.array(fit)
flux_tmp = flux / fit
flux = flux_tmp

# --- 6) save two-column ASCII: time, normalized flux ---
data = np.column_stack((time, flux))
np.savetxt("KIC 3975085 flux 2.txt", data)